In [3]:
Example_report = """- Despite a budget reduction, Q2 delivered an average reach that was 6% higher than planned.
- Doctor Discussion Guide was the top performing ad garnering over 56% of total impressions.This creative has consistently seen a majority of campaign delivery since launch on 1/1.
- Visits from reach optimized mixed placement ads decreased 31% QoQ as spend decreased and assets in market were paused as of 5/17.
- The Doctor Discussion Guide (DDG) ad continued to drive the majority of visits from mixed placement throughout Q2 (as it has since Q4 2021) at 31k, accounting for 70% of the mixed placement visits this quarter. The First of Its Kind ad replaced the Newly Diagnosed ad in Q2 as the second largest traffic driver from the group driving 8k visits.
- Looking Ahead: The Wunderman Thompson (WT) and Carat teams have aligned on removing Doctor Discussion Guide creative moving into Q3 to better diversify impression share across new creative and to increase visibility of different messages within the rotation.
"""

#Quarterly Impressions: 
# Campaign list, Campaign start/end date, Campaign impressions day wise/week wise, visits, Downloads

# within a quarter, identify the percentage distribution of Campaigns




Example_chart = """Paid Social Facebook/Instagram - Reach Optimized

Quaterly Impressions:
Week of Actualdate [2022], Doctor Discussion Guide, First Of Its Kind - 2021, Live In Your Moment-2021, MP New 125K Claim, Newly Diagnosed-2021
Mar 27, 400, 100, 20, 180, 10
Apr 3, 380, 120, 35, 100, 10	
Apr 10, 390, 115, 65, 55, 10	
Apr 17, 420, 110, 35, 70, 10			
Apr 24, 410, 180, 20, 85, 10		
May 1, 490,  100, 15, 105, 10	
May 8, 480,	120, 15, 120, 10
May 15, 370, 90, 15, 90, 10

Q2 Creative Performance:
Ad Description, Impressions, % of IMP
Doctor Discussion Guide, 58391183, 56%
First of its Kind - 2021, 18520560, 18%
MP New 125K Claim, 15768707, 15%
Live In Your Moment - 2021, 7109039, 7%
Newly Diagnosed - 2021, 4055879, 4%

Mixed Placements Creative Comparison - Daily Visits:
Date, Doctor Discussion Guide, First of Its Kind, Living in Your Moment, 125K Claim, Newly Diagnosed
1-Apr-22, 853, 521, 1175, 987, 714
8-Apr-22, 638, 482, 796, 1043, 1159
15-Apr-22, 927, 645, 432, 579,	1121
22-Apr-22, 419, 1198, 754, 503, 962
29-Apr-22, 1035, 764, 821, 1187, 593
6-May-22, 462, 947, 685, 551, 1016
13-May-22, 1142, 473, 938, 809, 725
20-May-22, 0, 0, 0, 0, 0	
27-May-22, 0, 0, 0, 0, 0	
3-Jun-22, 0, 0, 0, 0, 0	
10-Jun-22, 0, 0, 0, 0, 0	
17-Jun-22, 0, 0, 0, 0, 0	
24-Jun-22, 0, 0, 0, 0, 0	

"""
Csv_data_description = """Here is the column name and Description of the CSV Data
InvoiceNo	A unique identification number assigned to each transaction. (Numeric)
StockCode	A unique identification code assigned to each product sold by the retailer. (Numeric)
Description	A brief description of the product sold. (Text)
Quantity	The number of units of the product sold in each transaction. (Numeric)
InvoiceDate	The exact date and time when the transaction occurred. (Date/Time)
UnitPrice	The price per unit of the product sold. (Numeric)
Country	The country where the customer resides. (Text)
"""

In [4]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

import os
from dotenv import load_dotenv
import asyncio
import json
import re
import nest_asyncio
nest_asyncio.apply()
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
import traceback
import sys
import copy
import re
import subprocess

load_dotenv()

True

In [5]:
gpt_4_azure_ = AzureChatOpenAI(
    openai_api_base="url",
    openai_api_version="2023-05-15",
    deployment_name="gpt-4-turbo",
    openai_api_key="api_key",
    openai_api_type="azure",
    temperature=0.0,
    model_version="1106-Preview",
)



/Users/davidzoe/anaconda3/envs/test/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:163: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/Users/davidzoe/anaconda3/envs/test/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:171: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updating https://openai-newpage-canada.openai.azure.com/ to https://openai-newpage-canada.openai.azure.com/.
  warnings.warn(


In [6]:
#Ibrance Chart Data

#Page 4 - 5
Branded_Media_Executive_Summary = """Branded Media Executive Summary

Expenses
Q2-21:4358$, Q3-21:6397$, Q4-21:10884$, Q1-22:5037$, Q2-22:4282$(Display-1229$:12% descrease from last quater, Social-1320$:37% descrease from last quater, Search-1733$11% increase from last quater)

Quater, Display Clicks, Social impresions, Social clicks, Search Clicks
Q2-21, 522000, 190000000, 393000, 324000
Q3-21, 1138000, 263000000, 598000, 419000
Q4-21, 833000, 380000000, 799000, 558000
Q1-22, 440000, 263000000, 557000, 525000
Q2-22, 292000, 181000, 611000, 515000

Visits by traffic Source
Quater, SEM, Paid Social, Display digital video, DAR Digital video, Non-paid, Display banners, All video placements
Q2-21, 270000, 24000, 173000, 0, 88000, 0, 173000
Q3-21, 338000, 341000, 260000, 0, 115000, 194000, 455000
Q4-21, 473000, 430000, 71000, 270000, 111000, 249000, 0
Q1-22, 470000, 27000, 67000, 112000, 79000, 146000, 0
Q2-22, 436000, 256000, 97000, 90000, 79000, 74000, 0"""


# Page 8 - 10
Paid_Search = """Paid Search

Quater, Downloads, Clicks, CPA, Spend
Q1 Total, 4312, 460113, 261.82$, 1128988$
Q2 Total, 6185, 439085, 200.69$, 1241290$
Difference % QoQ, +43%, -5%, -23%, +10%

Quater, Unbranded Treatment, Unbranded Condition, Branded, Unbranded Symptoms, Competitive, Unbranded Ambassador
Q2-21, 152000, 55000, 25000, 30000, 8000, 1000
Q3-21, 58000, 144000, 20000, 99000, 16000, 1000
Q4-21, 85000, 214000, 25000, 129000, 19000, 1000
Q1-22, 97000, 182000, 28000, 145000, 18000, 2000
Q2-22, 126000, 152000, 37000, 104000, 15000, 3000

SEM Download Rate
Name, Q2 2021, Q3 2021, Q4 2021, Q1 2022, Q2 2022
Unbranded Treatment, 0.12%, 0.52%, 1.48%, 1.30%, 0.77%
Unbranded Condition, 1.41%, 1.72%, 1.80%, 1.52%, 5.49%
Branded, 2.49%, 2.22%, 1.30%, 1.89%, 1.36%
Unbranded Symptoms, 1.28%, 1.17%, 1.20%, 1.16%, 1.87%
Competitive, 0.38%, 0.33%, 0.31%, 0.27%, 0.26%
Unbranded Ambassador, 0.00%, 0.16%, 0.07%, 0.05%, 0.20%
Total, 0.74%, 1.31%, 1.49%, 1.33%, 2.71%"""


# Page 13 - 15
Paid_Social_Facebook_Instagram = """Paid Social Facebook/Instagram - Reach Optimized

Global:

Quaterly Impressions:
Week of Actualdate [2022], Doctor Discussion Guide, First Of Its Kind - 2021, Live In Your Moment-2021, MP New 125K Claim, Newly Diagnosed-2021
Mar 27, 400, 100, 20, 180, 10
Apr 3, 380, 120, 35, 100, 10	
Apr 10, 390, 115, 65, 55, 10	
Apr 17, 420, 110, 35, 70, 10			
Apr 24, 410, 180, 20, 85, 10		
May 1, 490,  100, 15, 105, 10	
May 8, 480,	120, 15, 120, 10
May 15, 370, 90, 15, 90, 10

Q2 Creative Performance:
Ad Description, Impressions, % of IMP
Doctor Discussion Guide, 58391183, 56%
First of its Kind - 2021, 18520560, 18%
MP New 125K Claim, 15768707, 15%
Live In Your Moment - 2021, 7109039, 7%
Newly Diagnosed - 2021, 4055879, 4%

Mixed Placements Creative Comparison - Daily Visits:
Date, Doctor Discussion Guide, First of Its Kind, Living in Your Moment, Newly Diagnosed, 125K Claim
1-Apr-22, 853, 521, 1175, 987, 714
8-Apr-22, 638, 482, 796, 1043, 1159
15-Apr-22, 927, 645, 432, 579,	1121
22-Apr-22, 419, 1198, 754, 503, 962
29-Apr-22, 1035, 764, 821, 1187, 593
6-May-22, 462, 947, 685, 551, 1016
13-May-22, 1142, 473, 938, 809, 725
20-May-22, 0, 0, 0, 0, 0	
27-May-22, 0, 0, 0, 0, 0	
3-Jun-22, 0, 0, 0, 0, 0	
10-Jun-22, 0, 0, 0, 0, 0	
17-Jun-22, 0, 0, 0, 0, 0	
24-Jun-22, 0, 0, 0, 0, 0	

Spanish:

Quaterly Impressions
Date, Doctor patient dialogue spanish, Talk on diagnosis spanish
Mar 27, 32000, 65000
Apr 3, 33000, 61000
Apr 10, 53000, 49000
Apr 17, 69000, 28000
Apr 24, 70000, 24000
May 1, 70000, 30000
May 8, 70000, 28000
May 15, 55000, 24000

Q2 Creative Performance
Ad Description, Impressions, % of IMP
Talk on Diagnosis Spanish, 8186357, 69%
Doctor Patient Dialogue Spanish, 3620856, 31%"""


# Page 17 - 18
Paid_Social_Twiter = """Paid Social Twiter - Reach Optimized

KPI: Monthly Reached
Planned: 30%
Actualized: 31%
April: 38%, May: 24%, June: - 

Q2 Creative Performance:
Ad Description, Impressions, % of IMP
Ambassador in the Moment 2022, 3354294, 21%
Talk to Your Doctor - 3.2.22, 3333837, 21%
BRANCE Answers 2022, 3313298, 21%
#1 Prescribed 2021, 3070022, 19%
Living in the Moment - 2021, 3056845, 19%

Twitter Ad Visits QoQ:
Quater, Talk to Your Doctor, #1 Prescribed, Living in the Moment, IBRANCE Answers, Ambassador In the Moment
Q4 2021, Campaign not started, 18486, 5533, Campaign not started, Campaign not started
Q1 2022, 781, 6379, 6267, 1308, 1041
Q2 2022, 1235, 1201, 1168, 1021, 718

Twitter Page Metrics:
Quarter, Profile Visits, Mentions, Impressions, Followers, % Follower Growth
Q4 2021, 39k, 21, 16.4k, 200, N/A
Q1 2022, 52k, 27, 13.3k, 362, 81%
Q2 2022, 22k, 8, 5.7k, 433, 20%"""


# Page 20 - 21
Paid_Social_Facebook = """Paid Social Facebook - Visit Optimized
CPC Benchmark $1.83 CPC

CPC:
Q2-21:1.83$, Q3-21:1.81$, Q4-21:2.53$, Q1-22:2.44$, Q2-22:1.04$

Clicks:
Q2-21:201749, Q3-21:187237, Q4-21:166864, Q1-22:223494, Q2-22:418040

Q2 Creative Performance:
Ad Description, Impressions, Clicks, CTR, CPC
Caregiver Support Version A, 30857002, 312363, 1.01%, $1.00
Patient Guides, 7467183, 81238, 1.09%, $1.11
Ibrance Answers 2021, 2902200, 21231, 0.73%, $1.19
Trial Results - 2021, 307540, 3208, 1.04%, $1.19

National visit Optimized
Metrics, Q2 2022, Q1 2022, QoQ
Visits, 170765, 149402, 14%
Pages/Visit, 1.07, 1.10, -3%, 3%
Bounce Rate, 92%, 89%
Avg. Visit Duration, 00:09, 00:13, -29%
Download Rate, 1.52%, 2.15%, -29%"""


# Page 23 - 24
Paid_Social_Local_Facebook = """Paid Social Facebook Traffic local
CPC Benchmark $2.99 CPC

CPC:
Q2-21:2.99$, Q3-21:2.55$, Q4-21:3.15$, Q1-22:4.17$, Q2-22:1.14$

Clicks:
Q2-21:26629, Q3-21:42055, Q4-21:66266, Q1-22:37395, Q2-22:53499

Q2 Creative Performance:
Ad Description, Impressions, Clicks, CTR, CPC
Caregiver Support Image Link Ad, 3858988, 40624, 1.05%, $1.11
Patient Guides Image Link Ad, 594990, 5990, 1.01%, $1.20
Doctor Discussion Guide Video Link Ad, 423092, 4075, 0.96%, $1.30
IBRANCE Answers - 2021 Video Link Ad, 265694, 1970, 0.74%, $1.32
Trial Results - 2022 Video Link Ad, 74786, 840, 1.12%, $1.22


General
Metrics, Q2 2022, Q1 2022, QoQ,
Visits, 13.4k, 18.9k, -75%
Pages/Visit, 1.08, 1.11, -3%
Bounce Rate, 92%, 89%, 4%
Avg. Visit Duration, 00:07, 00:13, -43%
Download Rate, 1.6%, 1.8%, -12%

HCP-Aligned
Metrics, Q2 2022, Q1 2022, QoQ
Visits, 10.4k, 5.5k, 86%
Pages/Visit, 1.07, 1.20, -11%
Bounce Rate, 92%, 88%, 5%
Avg. Visit Duration, 00:10, 00:22, -57%
Download Rate, 1.4%, 1.7%, -15%"""


# Page 25 - 26
Paid_Social_Hispanic_Facebook = """Paid Social Facebook Traffic Hispanic local
CPC Benchmark $2.52 CPC

CPC:
Q1-22:1.52$, Q2-22:1.69$

Clicks:
Q1-22:19600, Q2-22:14942

Q2 Creative Performance:
Ad Description, Impressions, Clicks, CTR, СРС
Talk on Diagnosis Spanish, 1490223, 9424, 0.63%, $1.65
Doctor Patient Dialogue Spanish SD, 925914, 5518, 0.60%, $1.75

Talk on Diagonisis
Metrics, Q2 2022, Q1 2022, QoQ,
Visits, 3.6k, 3.9k, -8%
Pages/Visit, 1.06, 1.07, -1%
Bounce Rate, 93%, 90%, 2%
Avg. Visit Duration, 00:14, 00:20, -31%
Download Rate, 0.05%, 0.28%, -80%


Doctor Patient Dialogue
Metrics, Q2 2022, Q1 2022, QoQ
Visits, 2k, 5.7k, -65%
Pages/Visit, 1.05, 1.07, -2%
Bounce Rate, 93%, 90%, 3%
Avg. Visit Duration, 00:12, 00:15, -19%
Download Rate, 1.03%, 1.61%, -36%"""


# Page 28
Digital_Video = """Digital Video - DAR

Total
Metrics, Q2 2022, Q1 2022, QoQ
Visits, 97k, 67k, 44%   
Pages/Visit, 1.20, 1.23, -2%   
Bounce Rate, 83%, 82%, 2%   
Avg. Visit Duration, 00:18, 00:15, 21%  
Download Rate, 0.01%, 0.00%, 456%   

Working it
Metrics, Q2 2022
Visits, 24k
Pages/Visit, 1.37
Bounce Rate, 83%
Avg. Visit Duration, 00:14
Download Rate, 0.00%

In this Together
Metrics, Q2 2022
Visits, 71k
Pages/Visit, 1.15
Bounce Rate, 87%
Avg. Visit Duration, 00:20
Download Rate, 0.01%"""


# Page 30 - 31
Digital_Video_Non_DAR = """Digital Video - Display Digital Video(Non-DAR)
CPC Benchmark $0.42 
CRT Benchmark 4.13%

CPC:
Q2-21:0.34$, Q3-21:0.27$, Q4-21:0.72$, Q1-22:0.51$, Q2-22:0.55$

Clicks:
Q2-21:315961, Q3-21:871889, Q4-21:433947, Q1-22:310619, Q2-22:175919

Q2 Creative Performance:
Creative Concept, Click Through Rate, Clicks, Impressions
Moments_Working It, 3.25%, 91102, 2798871
Moments In This Together, 3.17%, 84817, 2677939
  
Total
Metrics, Q2 2022, Q1 2022, QoQ
Visits, 73k, 145k,-50%
Pages/Visit, 1.44, 1.35,6%
Bounce Rate, 69%, 75%,-8%
Avg. Visit Duration, 00:15, 00:13,13%
Download Rate, 0.01%, 0.00%,429%

Working it
Metrics, 4/28 - 6/30
Visits, 25k
Pages/Visit, 1.50
Bounce Rate, 66%
Avg. Visit Duration, 00:20
Download Rate, 0.01%

In this Together
Metrics, 4/28-6/30
Visits, 23k
Pages/Visit, 1.49
Bounce Rate, 65%
Avg. Visit Duration, 00:20
Download Rate, 0.02%"""


# Page 34 - 35
Display_Banners = """Display Banners
Branded Health: CPC Benchmark $15.52 CRT Benchmark 0.13%

CPC:
Q2-21:17.00$, Q3-21:13.99$, Q4-21:15.88$, Q1-22:15.47$, Q2-22:14.05$

Clicks:
Q2-21:62615, Q3-21:80311, Q4-21:88745, Q1-22:68181, Q2-22:69362

Q2 Creative Performance:
Creative Concept, Click Through Rate, Clicks, Impressions
Unbranded 2019 Stories V2, 0.15%, 32385, 22123392
Savings and Support 2020, 0.14%, 20136, 13985970
Unbranded WebMD, 0.10%, 6845, 7005942
Unbranded Heritage. 125k HTML Banner, 0.14%, 3223, 2257695
MOA 2020, 0.12%, 1719, 1461305
Share Your Story 2022 - Health, 0.09%, 677, 795165
Efficacy 2021, 0.11%, 808, 736535
Priscilla 2022 - Health, 0.09%, 630, 692621
Amassadors 2021, 0.10%, 527, 524918
Moments 2020, 0. 16%, 826, 506755
Savings_and_Support_2020_Health, 0.14%, 492, 359966
MOA_2020: Health, 0.12%, 426, 359832
Unbranded_2019_Stories_Health, 0.12%, 438, 357762
Ambassadors 2021, 0.08%, 230, 276095
  

Branded Non-Endemic: CPC Benchmark $4.07 CRT Benchmark 0.13%

CPC:
Q2-21:2.20$, Q3-21:2.77$, Q4-21:6.32$, Q1-22:2.92$, Q2-22:3.30$

Clicks:
Q2-21:143288, Q3-21:168245, Q4-21:249179, Q1-22:61680, Q2-22:46932

Q2 Creative Performance:
Creative Concept, Click Through Rate, Clicks, Impressions
Ambassador V2, 0.13%, 31889, 24315076
Savings and Support 2020, 0.07%, 6276, 8958064
Unbranded Heritage 125K, 0.06%, 1412, 2314514
Spanish 2021 - Lifestyle, 0.06%, 1376, 2308261
Unbranded Stories 2019 V2, 0.09%, 1811, 1938393
MOA 2020, 0.07%, 1092, 1589261
Spanish_Website_Generations_2022 _Lifestyle, 0.08%, 1065, 1265054
Priscilla 2022 - Lifestyle, 0.05%, 521, 1033954
Share Your Story 2022 - Lifestyle, 0.06%, 456, 788771
Efficacy 2021, 0.07%, 491, 754284
Ambassadors 2021, 0.06%, 376, 631555
Moments 2020, 0.03%, 167, 566900

Display Visits QoQ

Quater, Visits
Q2-21, 159000
Q3-21, 192000
Q4-21, 250000
Q1-22, 109000
Q2-22, 89000

Video Start and Start rate for Display QoQ

Quater, video starts, video start rate
Q2-21, 1475, 0.93%
Q3-21, 1567, 0.82%
Q4-21, 1188, 0.47%
Q1-22, 1466, 1.35%
Q2-22, 1387, 1.56%"""

Ibrance_website = """Ibrance Website
Title, Count
I am a caregiver of a patient considering IBRANCE (palbociclib), 3
I am considering taking IBRANCE (palbociclib), 9
I am a caregiver of a patient taking IBRANCE (palbociclib), 10
I am taking IBRANCE (palbocicio), 47
None of the above, 121

Traffic Source**
• SEM - 77%
• Paid Social - 10%
• Banner - 7%
• DAR - 4%
• Non-DAR - 2%

Would you willing to answer a few Questions about your experience?
• No thanks 28%
• Yes, I will give feedback - 72%

How likely are you to recommend this website to other patients and caregivers?
All Users - 6.9
Targeted Audience - 7.5

Overall, how satisfied are you with this website?
All Users - 5.2
Targeted Audience - 5.3

It was easy to find what I was looking for on this website.
All Users - 5.4
Targeted Audience - 5.4

How likely are you to recommend this website to other patients and caregivers?
type, 0,1,2,3,4,5,6,7,8,9,10
All Users,8,6,4,4,4,28,7,12,18,11,51
Targeted Audience,1,3,2,1,0,7,7,2,6,8,8,18

Why are you dissatisfied with this website? Choose all that apply

type, Targeted Audience, All Users
Other issue using the site,4,12
The content was not relevant to me,2,11
The site was difficult to navigate,3,7
The content was difficult to understand,5,5
The site took too long to load,1,2

-What can we do to make it easier for you to find what you were looking for on this website? Choose all that apply.

type, Targeted Audience, All Users
Other issue using the site,3,12
Provide more relevant content about my disease,4,9
Add search functionality,3,8
Improve site navigation,0,6
Provide more relevant content about my treatment,3,4
"""

Ibrance_com_kpi = """Ibrance.com KPI
Metrics, Q2 2022, Q1 2022, QoQ, Q2 2021, Projections & Benchmarks*
Visits, 1029758, 1153308, -11%, 957624, 1M-1.2M
Pages/Visit, 1.2, 1.2, 0%, 1.2, 1.2
Avg. Visit Duration, 30, 30, 0%, 29, 30
Bounce Rate, 81%, 80%, 1%, 71%, 75%
Downloads, 19719, 15015, 31%, 6717, 7500
Download Rate, 1.9%, 1.3%, 47%, 0.7%, 0.7%



Download Volume and Rate Over Time
Quater, Total Downloads, Download Rate
02 2021, 6717, 0.70%
Q3 2021,10306, 0.71%
Q4 2021, 13348, 0.83%
Q1 2022 , 15015, 1.30%
Q2 2022, 19719, 1.91%

English Downloads

• Fulvestrant Dosing Tracker - 8%
• Doctor Discussion Guide Considering IBRANCE - 8%
• Doctor Discussion Guide Taking IBRANCE- 8%
• Al Dosing Tracker - 11%
• Patient Brochure - 23%
" Caregiver Guide - 25%
• Newly Diagnosed Tips - 17%

Spanish Downloads

• Bilingual Guide - 41%
• Spanish Al Dosing Tracker - 6%
• Spanish Caregiver Guide - 16%
• Spanish Doctor Discussion Guide Considering - 6%
• Spanish Doctor Discussion Guide Taking - 10%
• Spanish Fuldestrant Tracker - 5%
• Spanish Important Facts - 16%

Quater, Downloads, Download Rate
Q4 2020, 201, 0.53%
Q1 2021, 338, 1.05%
Q2 2021, 219, 1.28%

Site Registrations QoQ
Quater, Total Unique Visitors to Registration Page, % of Unique Visitors that Visit Registration Page, % of Unique Visitors to the Registration Page that Visit Registration Complete Page
Q2 2021, 5603, 0.68%, 11.78%
Q3 2021, 4216, 0.34% 13.69%
Q4 2021, 5893, 0.41%, 9.77%
Q1 2022, 21349, 2.11%, 4.93%
Q2 2022, 9340, 1.02%, 6.07%
"""


#Ibrance Report Summary
Branded_Media_Executive_Summary_Report = """
Overall Branded spend decreased by 15% QoQ as Programmatic and Social tactics were paused mid-May. Endemic and Search activity remained live for the full quarter.
- Display: Spend decreased 12% QoQ as Programmatic and Non DAR Video activity was paused mid-May, driving a 34% decrease in clicks.
    - Due to the efficiency of non-DAR Video, overall clicks decreased disproportionally to spend
- Social: Despite activity being paused mid-May, clicks increased 10% QoQ as CPCs were lower within the Traffic campaigns.
- Search: Clicks remained flat QoQ despite increase in spend due to higher competition and CPCs in Q2
- Overall visits to the site declined by 11% QoQ from 1.1M to 1M, with SEM continuing to drive the highest share of traffic at 42% of total visits. Paid search visits declined QoQ by 7%.
- Paid social continued to drive the second largest traffic volume, accounting for 25% of total. Overall volume from the channel declined QoQ by 8%. Traffic from the visit optimized buy increased 12% QoQ while reach optimized decreased 39% QoQ.
- DAR digital video drove the third largest share of traffic this quarter at 97k visits, accounting for 9% of total. Display banners drove the fourth strongest volume. Display digital video traffic was understated but tagging was resolved on 4/28.
"""
Paid_Search_Report = """
Overall:
- Even with a slight 10% budget increase QoQ due to budget shifts, search volume remained relatively steady within clicks and impressions
- Competition within the marketplace arose QoQ, CPCs also inflated by 13% from $2.97 to $3.36
    - Kisqali HCP re-entered the competitive space last quarter, after dropping out from the space back in Q1 for the Branded and Condition pathways. This attributed to the increased CPC last quarter

Performance:
- As the Condition and Symptom pathways shifted into an auction time bidding strategy in Q1, both pathways continue to take the lead in overall traffic
    - Although both pathways experienced heavy competition in Q2, the Symptoms campaigns continued to present CPC efficiencies of $2.71 and a 10% growth in CTR, while the Condition campaigns displayed $2.80 in CPC efficiencies and a 17% increase in CTR
- The Branded pathway remains the most efficient pathway with a steady SoS of 88%
    - SoS experienced a slight dip last quarter due to increased traffic over on Bing

HVA:
- Overall, HVA performance efficiency has improved significantly QoQ going into Q2'22. CPA was lower (from $261.82 to $200.69), which was due to improvements within the automated bidding strategies along with budget allocation shifts that were made in Q1
    - Additionaly, due to budget allocation shifts QoQ downloads significantly increased by 43%
- The Condition pathway drove the most downloads at 3,007 while the Symptom pathway drove 763 downloads in Q2

- Although visits from SEM declined by 7% QoQ from 470k to 436k, the channel remained the leading traffic driver in Q2. Site side engagement from SEM stayed relatively flat QoQ
- The Branded and Unbranded Treatment pathways had the largest increases in visit volume QoQ at 36% and 30%, respectively. Unbranded Condition continued to drive the majority of visit volume from the channel.
- Video start rate from SEM declined by 25% QoQ from 3.72% to 2.79%. Video start rate for SEM has increased every quarter since HVA targeting was implemented aside from this quarter. Video start rate from HVA targeting SEM pathways specifically also declined 20% QoQ
- Despite the decline in video starts, SEM still drove 74% of total video starts for the quarter representing 12.1k of the 17.9k, with starts of over 30 different videos on site.
- Despite the 7% decrease in visits QoQ from SEM, download volume increased 88% from 6.2k to 11.8k and rate increased 103% from 1.33% to 2.71%.
- The increase in downloads was mainly driven by a 202% increase in downloads from Unbranded Condition (from 2.7k to 8.3k). Unbranded Symptoms also saw an increase in download volume by 16%.
- With an increase in downloads, Unbranded Condition had the highest download rate of the SEM pathways during Q2 (and higher than any pathway's rate in the past year).
- SEM drove 60% of total download volume in Q2, driving downloads of both the Patient Brochure (30%) and Newly Diagnosed Tips (27%).
"""

Paid_Social_Facebook_Instagram_report = """
- Despite a budget reduction, Q2 delivered an average reach that was 6% higher than planned.
- Doctor Discussion Guide was the top performing ad garnering over 56% of total impressions. This creative has consistently seen a majority of campaign delivery since launch on 1/1.
- Visits from reach optimized mixed placement ads decreased 31% QoQ as spend decreased and assets in market were paused as of 5/17.
- The Doctor Discussion Guide (DDG) ad continued to drive the majority of visits from mixed placement throughout Q2 (as it has since Q4 2021) at 31k, accounting for 70% of the mixed placement visits this quarter. The First of Its Kind ad replaced the Newly Diagnosed ad in Q2 as the second largest traffic driver from the group driving 8k visits.
- Looking Ahead: The Wunderman Thompson (WT) and Carat teams have aligned on removing Doctor Discussion Guide creative moving into Q3 to better diversify impression share across new creative and to increase visibility of different messages within the rotation.

- With a media pause in May, Q2 exceeded the planned benchmark by an average of 12%.
- Talk on Diagnosis creative continued to outperform Doctor Patient Dialogue, garnering 69% of total campaign impressions.
"""

Paid_Social_Twiter_Report = """
- Due to the media pause in May with spend down 53% and also a 20% increase in CPM, this campaign saw a 53% decrease in impressions QoQ.
- There was also substantial competition in the market which led to increased CPMs the week prior to pausing which caused a decrease in impressions.
- Performance across creatives was virtually identical.
- A dip can be seen on 5/16 when the campaigns were paused, however, the Added Value campaign was reactivated to spend in full through the end of May
    - Added Value was generated from Q1 investment.
- isits from the Twitter ads were down 81% QoQ, following a 48% decline in volume from Q4 to Q1, due to a decrease in budget and an increase in CPMs. Twitter ads collectively drove 2.3k visits in Q2 and 12k in Q1, and site side engagement remained steady.
- The Talk to Your Doctor ad continued to have the highest download rate of the Twitter ads performing well above the reach optimization average download rate at 0.32%. This was also the only ad that increased in visit volume QoQ.
- Twitter followers grew 21% QoQ although they increased more rapidly from Q4 to Q1. This could be due to the longevity of the page and or no new drivers to the page in Q2.
"""

Paid_Social_Facebook_Report = """
- Despite the Q2 budget cut, QoQ spend remained flat as there were budget shifts from March into Q2
    - Impressions increased 101% which led to an 87% increase in overall Link Clicks.
    - This is largely due to our shift to optimize to Link Clicks rather than LPV, which lowered CPC and CPM roughly 50% allowing us to be overall more efficient.
- Static creatives continue to outperform our video ads. This is likely due to a clearer and more succinct message in the static ads compared to video, where ads are lengthier and do not feature a CTA within the first few frames.
- Visit volume from the national traffic optimized rotation increased 14% from 149k to 170k QoQ with a slight decline in site side engagement metrics. The download rate across the buy declined due to a decrease in download volume across all ads, but especially the Patient Guides ad (-51% QoQ).
- Seeing a 54% visit volume increase QoQ, the Caregiver ad brought in over 75% of all visits from this buy. This ad continues to be a top performer across all traffic optimized rotations.
- The Trial Results ad continues to drive exceptional site side engagement both within the national buy (1.20 pages per visit, 72 second average visit duration, and 76% bounce rate) and within the local buy
- Looking Ahead: The national traffic optimized rotation was not planned to run in Q3.
"""

Paid_Social_Local_Facebook_Report = """
- Similar to the National Traffic campaign, shifting to link click optimizations in Q2 improved CPCs and CPMs roughly 45% each allowing for improved efficiency
- The launch of HCP markets in March drove strong performance into Q2 and is the primary driver of the overall improved performance.
- The campaign saw a 43% increase in link clicks QoQ (53.5K vs 37.4K). CPC for this quarter was 46.5% more efficient than Q1 at $1.14.
- Caregiver Support creative is garnering the highest clicks and most efficient CPC of all creatives.
- Although the general markets drove more visit volume, the visits declined 75% QoQ, while the HCP-aligned markets saw an 86% increase in visits from Q1. Visits from the combined markets stayed relatively flat QoQ.
- The Caregiver ad continued to be a top performer across both markets; seeing an increase in visits of 22% for the General markets, representing 86% of total visits, and a 188% increase for the HCP-aligned markets, representing 74% of total.
- The Trial Results ad continued to drive the strongest site side engagement metrics across both market groupings.
- Looking Ahead: Moving into Q3, creative rotations will continue to differ based on specific performance by ad across the two market groupings.
"""

Paid_Social_Hispanic_Facebook_Report = """
- CPCs improved QoQ by 43% ($1.69 vs $2.52) however overall link clicks were down 24% and impressions were down 6% as spend decreased by 50%.
- Talk on Diagnosis creative garnered more impressions, a higher CTR and a more efficient CPC than the Doctor Patient Dialogue creative.
- Due to the media pause on 5/17, the Spanish traffic optimized visits decreased 47% QoQ from 9.7k to 5.6k visits and download volume was down 78%.
- While the Talk on Diagnosis ad drove the most visits this quarter of the two ads with 3.6k visits, the Doctor Patient Dialogue ad drove stronger download volume and rate. The Bilingual Guide represented over 70% of the downloads from the Doctor Patient Dialogue ad and remains the most downloaded Spanish document.
- Talk on Diagnosis ad drove the most video starts of the two ads at 36, for a start rate of 0.93%.
"""

Digital_Video_Report = """
- Visits from the DAR video buy increased 44% QoQ from 67k visits in Q1 to 97k visits in Q2. This increase allowed for this channel to become a leading traffic driver for the quarter, although visits are not the primary KPI for this buy.
- GRPs for this buy increased 33% QoQ, increasing the impressions for the buy, which could have attributed to the visit volume increase.
- Overall, Working It creative drove stronger pages per visit on site and lower bounce rate, while the In This Together creative drove more visit volume to the site with a higher average visit duration.
"""

Digital_Video_Non_DAR_Report = """
Branded Video:
- Campaign paused on 5/17 due to budget cut
- CPC was consistent from Q1 to Q2 but remained slightly higher vs benchmark at $.55 vs $.42
- Demo was the top-performing tactic with 13% more efficient CPC than Q2 and drove 55% of total clicks
- Campaign launched with Newsweek on 4/11 as a new partner to align with DE&| investment goals
    - Its CPC slightly underperformed against the benchmark -This is expected and can be attributed to the short flighting period.
    - Additional time is needed to perform and optimize toward the CPC benchmark
- Despite the short flight we saw strong delivery results - Newsweek was responsible for 45% of the total Video Q2 spend and generated 35% of total impressions in Q2

Creative:
- Moments_Working It and Moments_In This Together video creatives continued to show strong performance with high CTRs compared to other digital channels
Due to tagging dropping off the Working It creative when In This Together was rotated into market at the end of Q1, total visits for the quarter are understated for Q2. The tagging issue was resolved as of 4/28 and WT and Carat teams aligned on new QA process to eliminate future tracking errors.
- Additionally, due to budget limitations, this buy was de-prioritized in mid-May, so both creatives were rotated out of market.
- Performance between the two creatives remained very similar overall.
- Looking Ahead: Moving into Q3, the programmatic video buy will include more placements with a DEI focus.
"""

Display_Banners_Report = """
Branded Health:
- CPCs improved ~9% QoQ driven by *BDO and WebMD CPC efficiencies
- Remedy Health continued to be the top performing partner, outperforming both CPC & CTR benchmarks by ~42% 
- WebMD garnered strong results in Q2, outperforming CPC and CTR benchmarks
- BDO Lifestyle placement breakout launched on 4/4;
    - Placements outperformed Lifestyle in Q2, driving a CPC 37% more efficient and a slightly stronger click-through-rate (0.08% vs. 0.07 %)
    - BDO improvements attributed to ongoing backend optimizations, including site updates to improve viewability

Branded Non-Endemic:
- Paused on 5/17 due to budget cut
- Overall, the campaign performed efficiently, outperforming CPC benchmark
- Demo was the top-performing tactic, driving 58% of total clicks while yielding a CPC 12% more efficient than the Q2 average

Creative:
- The Unbranded 2019 stories V2 was the strongest performing creative within Health campaign
- Ambassador V2 was the strongest performing creative within Non-Endemic campaign
- Spanish Website Generations launched within Non-Endemic on 4/28
- New Pfizer logo updated banners Unbranded 2019 stories, MOA 2020, and Savings and Support banners were added into Health rotation on 6/28
- Visits from display decreased by 19% QoQ (following a 56% decrease from Q4 to Q1) due to lifestyle buy being paused mid-quarter. Engagement on site remained steady QoQ following an improvement going into 2022 for the channel.
- This quarter, the Ambassador banner (29k) led in overall traffic to the site, followed by Savings and Support (20k) and Unbranded Stories (19k). The MOA banner drove the bulk of visits last quarter but declined 91% QoQ, meanwhile the Ambassador banner increased over 4X QoQ.
- Although video start volume decreased 5%, the video start rate from display continued to increase QoQ at 16% from 1.35% to 1.56%. The Unbranded Stories and Ambassador banner drove the majority of video starts at 70%, but the Priscilla banner had the highest start rate at 8%.
- The new Spanish Generations banner drove the highest download rate of the display banners at 0.37% although volume was low.
"""

Ibrance_website_Report = """
- The survey was mainly shown to users coming from SE which represented 77% of total, followed by paid social 10% of total. Mobile represented over 75% of device exposure. These trends are reflective of overall site traffic.
- The Caregiver ad drove the majority of paid social survey responses, while the Unbranded Condition and Symptoms pathways drove 68% of the SEM survey responses.
- Although a quarter of the users agreed to provide feedback, only 19% responded to later questions. The "What best describes your situation" question allowed a target audience to be established as patients/caregivers who are taking/considering taking IBRANCE.* Of the 190 responses, 25% of users responded they are taking IBRANCE while 36% fell into the broader target audience.
- The Understanding Your Diagnosis page drove the strongest percentage of both who was served the survey and who responded, representing over 40% for both cases.
- Over 70% of the targeted audience would recommend the website while only 60% of all survey responders would recommend the website to other patients and caregivers, a score response over 7.
- Users are overall not very satisfied with the website, receiving a weighted score of 5.2 out of 10. There were no scores above 7.
- There were fewer responses to the "it was easy to find what I was looking for on this website" question, but users found it relatively difficult to find "what they were looking for" on the website with a score of 5.4 out of 10.
- 30 users responded to the question "Why are you dissatisfied with this website," the most common answer was "other issue using the site." Of the 9 people who answered "The content was not relevant to me," only one of them responded they were taking IBRANCE. Additionally, only 36% of respondents to this question were within the target audience.
- The most common answer for how to make it easier to use the site was "other issue using the site."
- Of the users who responded "provide more relevant content about my disease/treatment," more than half of them are the targeted audience*
- Quantity of responses dropped off after each question and there were no answers on the last question. WT to continue monitoring levels of engagement throughout various points of the survey.

"""

Ibrance_com_kpi_Report = """- Although visits to the site decreased 11% QoQ due to decreased budget, IBRANCE KPIs remain within benchmark aside from bounce rate.
- Site-side engagement metrics remained steady overall QoQ.
- Downloads improved 31% QoQ and 2X YoY due to a large lift in downloads, mainly driven from the Unbranded Condition SEM pathway (which was up 202% in volume QoQ).
- Download rate improved 47% QoQ and 170% YoY. This was in part due to the decrease in visits but primarily due to the increased download volume.
- QoQ, download volume increased 31% and download rate increased 46%. Downloads and rate have continued to improve each quarter since Q1 2021. There was a 194% and 173% increase YoY in downloads and download rate, respectively.
- Overall downloads were mainly driven by SEM (58%), followed by direct visits (20%), and paid social (16%).
- The Caregiver Guide became the most downloaded document, driving 25% of total, followed by the Patient Brochure at 23% and the Newly Diagnosed Tips sheet at 17%.
Taking
    - Downloads of the Caregiver Guide were driven 47% by SEM and 42% from direct visits.
The Resources to Download page drove the majority of downloads in Q2 at 55%, followed by the Getting Started page, which drove 17%, and finally, the Caregiver Resources page, which drove 14%.
- Spanish downloads saw a 35% decrease from Q1 with 219 downloads in Q2 compared to 338 in Q1. Despite the decrease in volume, download rate on the Spanish page increased 21% from 1.17% to 1.41% QoQ.
- Visits to the page continued to decline with a 47% dropQoQ from 29k to 15k, mainly driven by a decline in visits fron Spanish paid social, which decreased 44% QoQ, as well as a decline in display visits due to media budget decreases.
- This quarter, direct visits and SEO drove the majority of Spanish downloads, driving 40% and 24% respectively. Paid social was the third strongest driver at 17% with Doctor Patient Dialogue ad driving the majority.
- The Bilingual Guide remained the #1 downloaded document on the Spanish page, followed by the Caregiver Guide and Important Facts. Together, these made up over 70% of downloads on the page.
- In Q2, there was a dip in unique visitors to the registration page QoQ following the usual uptick in Q1 due to seasonality trends.
- The BRANCE Answers Facebook ads continued to drive the most volume of unique visitors to the Registration page, but the completion rate was low at 0.07%. These ads drove about 50% of the unique visitors to the page in Q2. The Twitter ad also drove a significant amount of visits to the page but no completions.
- Branded SEM keywords continued to drive the highest volume of unique visitors to the Registration Complete page at 262 and had the highest completion rate at 45%. SEO had the second highest volume and rate.
- Registration reporting from the database is still in queue to be reestablished following the PMAP migration.
"""

Paid_Social_National_buy = """
Mixed Placement Facebook & Instagram Reach Optimized:
- Despite a budget reduction, Q2 delivered an average reach that was 6% higher than planned.
- Doctor Discussion Guide was the top performing ad garnering over 56% of total impressions. This creative has consistently seen a majority of campaign delivery since launch on 1/1.
- Visits from reach optimized mixed placement ads decreased 31% QoQ as spend decreased and assets in market were paused as of 5/17.
- The Doctor Discussion Guide (DDG) ad continued to drive the majority of visits from mixed placement throughout Q2 (as it has since Q4 2021) at 31k, accounting for 70% of the mixed placement visits this quarter. The First of Its Kind ad replaced the Newly Diagnosed ad in Q2 as the second largest traffic driver from the group driving 8k visits.
- Looking Ahead: The Wunderman Thompson (WT) and Carat teams have aligned on removing Doctor Discussion Guide creative moving into Q3 to better diversify impression share across new creative and to increase visibility of different messages within the rotation.

- With a media pause in May, Q2 exceeded the planned benchmark by an average of 12%.
- Talk on Diagnosis creative continued to outperform Doctor Patient Dialogue, garnering 69% of total campaign impressions.

Twitter Reach Optimized:
- Due to the media pause in May with spend down 53% and also a 20% increase in CPM, this campaign saw a 53% decrease in impressions QoQ.
- There was also substantial competition in the market which led to increased CPMs the week prior to pausing which caused a decrease in impressions.
- Performance across creatives was virtually identical.
- A dip can be seen on 5/16 when the campaigns were paused, however, the Added Value campaign was reactivated to spend in full through the end of May
    - Added Value was generated from Q1 investment.
- Visits from the Twitter ads were down 81% QoQ, following a 48% decline in volume from Q4 to Q1, due to a decrease in budget and an increase in CPMs. Twitter ads collectively drove 2.3k visits in Q2 and 12k in Q1, and site side engagement remained steady.
- The Talk to Your Doctor ad continued to have the highest download rate of the Twitter ads performing well above the reach optimization average download rate at 0.32%. This was also the only ad that increased in visit volume QoQ.
- Twitter followers grew 21% QoQ although they increased more rapidly from Q4 to Q1. This could be due to the longevity of the page and or no new drivers to the page in Q2.

Facebook Visits Optimized:
- Despite the Q2 budget cut, QoQ spend remained flat as there were budget shifts from March into Q2
    - Impressions increased 101% which led to an 87% increase in overall Link Clicks.
    - This is largely due to our shift to optimize to Link Clicks rather than LPV, which lowered CPC and CPM roughly 50% allowing us to be overall more efficient.
- Static creatives continue to outperform our video ads. This is likely due to a clearer and more succinct message in the static ads compared to video, where ads are lengthier and do not feature a CTA within the first few frames.
- Visit volume from the national traffic optimized rotation increased 14% from 149k to 170k QoQ with a slight decline in site side engagement metrics. The download rate across the buy declined due to a decrease in download volume across all ads, but especially the Patient Guides ad (-51% QoQ).
- Seeing a 54% visit volume increase QoQ, the Caregiver ad brought in over 75% of all visits from this buy. This ad continues to be a top performer across all traffic optimized rotations.
- The Trial Results ad continues to drive exceptional site side engagement both within the national buy (1.20 pages per visit, 72 second average visit duration, and 76% bounce rate) and within the local buy
- Looking Ahead: The national traffic optimized rotation was not planned to run in Q3.
"""

Paid_Social_National_buy_chart = """
1. Paid Social Facebook&Instagram - Reach Optimized


Global:

Quaterly Impressions:
Week of Actualdate [2022], Doctor Discussion Guide, First Of Its Kind - 2021, Live In Your Moment-2021, MP New 125K Claim, Newly Diagnosed-2021
Mar 27, 400, 100, 20, 180, 10
Apr 3, 380, 120, 35, 100, 10	
Apr 10, 390, 115, 65, 55, 10	
Apr 17, 420, 110, 35, 70, 10			
Apr 24, 410, 180, 20, 85, 10		
May 1, 490,  100, 15, 105, 10	
May 8, 480,	120, 15, 120, 10
May 15, 370, 90, 15, 90, 10

Q2 Creative Performance:
Ad Description, Impressions, % of IMP
Doctor Discussion Guide, 58391183, 56%
First of its Kind - 2021, 18520560, 18%
MP New 125K Claim, 15768707, 15%
Live In Your Moment - 2021, 7109039, 7%
Newly Diagnosed - 2021, 4055879, 4%

Mixed Placements Creative Comparison - Daily Visits:
Date, Doctor Discussion Guide, First of Its Kind, Living in Your Moment, Newly Diagnosed, 125K Claim
1-Apr-22, 853, 521, 1175, 987, 714
8-Apr-22, 638, 482, 796, 1043, 1159
15-Apr-22, 927, 645, 432, 579,	1121
22-Apr-22, 419, 1198, 754, 503, 962
29-Apr-22, 1035, 764, 821, 1187, 593
6-May-22, 462, 947, 685, 551, 1016
13-May-22, 1142, 473, 938, 809, 725
20-May-22, 0, 0, 0, 0, 0	
27-May-22, 0, 0, 0, 0, 0	
3-Jun-22, 0, 0, 0, 0, 0	
10-Jun-22, 0, 0, 0, 0, 0	
17-Jun-22, 0, 0, 0, 0, 0	
24-Jun-22, 0, 0, 0, 0, 0	

Spanish:


Quaterly Impressions:
Date, Doctor patient dialogue spanish, Talk on diagnosis spanish
Mar 27, 32000, 65000
Apr 3, 33000, 61000
Apr 10, 53000, 49000
Apr 17, 69000, 28000
Apr 24, 70000, 24000
May 1, 70000, 30000
May 8, 70000, 28000
May 15, 55000, 24000

Q2 Creative Performance:
Ad Description, Impressions, % of IMP
Talk on Diagnosis Spanish, 8186357, 69%
Doctor Patient Dialogue Spanish, 3620856, 31%


2. Paid Social Twiter - Reach Optimized

Q2 Creative Performance:
Ad Description, Impressions, % of IMP
Ambassador in the Moment 2022, 3354294, 21%
Talk to Your Doctor - 3.2.22, 3333837, 21%
BRANCE Answers 2022, 3313298, 21%
#1 Prescribed 2021, 3070022, 19%
Living in the Moment - 2021, 3056845, 19%

Twitter Ad Visits QoQ:
Quater, Talk to Your Doctor, #1 Prescribed, Living in the Moment, IBRANCE Answers, Ambassador In the Moment
Q4 2021, Campaign not started, 18486, 5533, Campaign not started, Campaign not started
Q1 2022, 781, 6379, 6267, 1308, 1041
Q2 2022, 1235, 1201, 1168, 1021, 718

Twitter Page Metrics:
Quarter, Profile Visits, Mentions, Impressions, Followers, % Follower Growth
Q4 2021, 39k, 21, 16.4k, 200, N/A
Q1 2022, 52k, 27, 13.3k, 362, 81%
Q2 2022, 22k, 8, 5.7k, 433, 20%


3. Paid Social Facebook - Visit Optimized


CPC Benchmark - $1.83

CPC:
Q2-21:1.83$, Q3-21:1.81$, Q4-21:2.53$, Q1-22:2.44$, Q2-22:1.04$

Clicks:
Q2-21:201749, Q3-21:187237, Q4-21:166864, Q1-22:223494, Q2-22:418040

Q2 Creative Performance:
Ad Description, Impressions, Clicks, CTR, CPC
Caregiver Support Version A, 30857002, 312363, 1.01%, $1.00
Patient Guides, 7467183, 81238, 1.09%, $1.11
Ibrance Answers 2021, 2902200, 21231, 0.73%, $1.19
Trial Results - 2021, 307540, 3208, 1.04%, $1.19

National visit Optimized:
Metrics, Q2 2022, Q1 2022, QoQ
Visits, 170765, 149402, 14%
Pages/Visit, 1.07, 1.10, -3%, 3%
Bounce Rate, 92%, 89%
Avg. Visit Duration, 00:09, 00:13, -29%
Download Rate, 1.52%, 2.15%, -29%

"""

chart_sections = [Branded_Media_Executive_Summary, 
            Paid_Search, 
            Paid_Social_Facebook_Instagram, 
            Paid_Social_Twiter, 
            Paid_Social_Facebook, 
            Paid_Social_Local_Facebook, 
            Paid_Social_Hispanic_Facebook, 
            Digital_Video, 
            Digital_Video_Non_DAR, 
            Display_Banners,
            Ibrance_website,
            Ibrance_com_kpi
           ]
report_sections = [Branded_Media_Executive_Summary_Report, 
                   Paid_Search_Report, 
                   Paid_Social_Facebook_Instagram_report, 
                   Paid_Social_Twiter_Report, 
                   Paid_Social_Facebook_Report, 
                   Paid_Social_Local_Facebook_Report, 
                   Paid_Social_Hispanic_Facebook_Report, 
                   Digital_Video_Report, 
                   Digital_Video_Non_DAR_Report, 
                   Display_Banners_Report,
                   Ibrance_website_Report,
                   Ibrance_com_kpi_Report
]

individual_dict = [{"Chart":Paid_Social_National_buy_chart, "Report":Paid_Social_National_buy}]

actual_dict = [{"Chart":Branded_Media_Executive_Summary, "Report":Branded_Media_Executive_Summary_Report},
               {"Chart":Paid_Search, "Report":Paid_Search_Report},
               {"Chart":Paid_Social_Facebook_Instagram, "Report":Paid_Social_Facebook_Instagram_report}, 
               {"Chart":Paid_Social_Twiter, "Report":Paid_Social_Twiter_Report}, 
               {"Chart":Paid_Social_Facebook, "Report":Paid_Social_Facebook_Report}, 
               {"Chart":Paid_Social_Local_Facebook, "Report":Paid_Social_Local_Facebook_Report}, 
               {"Chart":Paid_Social_Hispanic_Facebook, "Report":Paid_Social_Hispanic_Facebook_Report}, 
               {"Chart":Digital_Video, "Report":Digital_Video_Report}, 
               {"Chart":Digital_Video_Non_DAR, "Report":Digital_Video_Non_DAR_Report}, 
               {"Chart":Display_Banners, "Report":Display_Banners_Report},   
               {"Chart":Ibrance_website, "Report":Ibrance_website_Report},   
               {"Chart":Ibrance_com_kpi, "Report":Ibrance_com_kpi_Report}   
]

In [7]:
Paid_Social_Current = """
CREATIVE_DESCRIPTION, CLICKS, IMPRESSIONS, MEDIA_COST, DCM_FLOODLIGHTS_UNIQUE_VISITS, VIDEO_VIEWS

BCG Traffic, 12.0, 2.0, 0.0000, 0.0, 0.0
Class Leadership 2023, 1325., 151677.0, 1305.8217, 4.0, 14399
Doctor Discussion Guide, 12322.0, 1119076.0, 12700.4159, 12.0, 202881.0
Financial Support 2022, 3333.0, 235334.0, 3402.1164, 3.0, 49794.0
I am Still Me Driving, 38572.0, 4053184.0, 38555.6432, 33.0, 670102.0
I am Still Me Gaming, 50400.0, 7138341.0, 52971.4538, 36.0, 1044314.0
Newly Diagnosed, 367.0, 52055.0, 370.9775, 3.0, 5640.0
Not Traditional Chemo, 673.0, 67812.0, 607.0957, 4.0, 9344.0
Real World Evidence, 14616.0, 1478048.0, 14793.5061, 26.0, 0.0
Ways to Connect, 2.0, 0.0, 0.0000, 0.0, 0.0
Webinar Driver, 31870.0, 3218559.0, 31554.6514, 66.0, 0.0
"""

### Report Generator

In [8]:
# Define your desired data structure.
class Report(BaseModel):
    Report: str = Field(description="It should be a quaterly report.")
report_parser = PydanticOutputParser(pydantic_object=Report)


In [467]:
#Few Shot Template

#     Your task is to generate a Quarterly report with the given data
# you will be given "Example report" Just for your understanding and reference
# And you will be given a Raw Report which has all the detail in it and CSV data of each chart which used to get the Raw Report. with these prepare a Detailed Insights Report.
# Use the numbers in your report while explaining it. 

# Format Instructions:
# {format_instructions}
# Strictly follow this instruction

# Example report:
# {report}
# This Example report is just for your reference, Don't use the "Example report" Anywhere while generating the report 

# Raw Report:
# {raw_report}

# CSV Chart Data:
# {chart_data}

# Remember to focus mostly on recent quater and compare this with it's past, And always give Numbers while comparing.
# Now begin to generate a quaterly report, with great insights.

#Zero Shot Template
# Your task is to generate a Quarterly report with the given data 
# With the Differnet chart data in "Textual Chart Data" which is textually formatted(Mostly CSV) you need to generate a quaterly report
# You should use the numbers in your report while explaining it. And make a detailed Insights points.
# And you should have atleast 5 points for each.

# Format Instructions:
# {format_instructions}
# Strictly follow this instruction

# Textual Chart Data:
# {chart_data}

# Explain all the textual representation(Mostly CSV data) of chart data under "Textual Chart Data" and correlate those chart data in "Textual Chart Data" to get deep insights from all POV. 
# your numbers should always match with actual number don't Hallucinate.

# Remember to focus mostly on recent quater and compare this with it's past
# Now begin to generate a quaterly report with "Textual Chart Data", don't forget to correlate the chart data to get deep insights and don't miss any information.



In [9]:
async def few_shot_summary_generator(Example_report, Chart_data, raw_report):
    response_schemas = [
        ResponseSchema(name="Report", description="It Should Be a string."),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    # format_instructions = output_parser.get_format_instructions()
    format_instructions = report_parser.get_format_instructions()
    
    template = """
You are an AI Assistant helping user to generate a Quarterly report with the given data.
The given data are,
    Example report - it is a part of a Quaterly report, just given to you for your reference and understanding 
    Raw Report - this has mostly all the detail you need to write a quaterly report
    CSV Chart Data - is where you can find all the raw data(CSV formatted), this data is used while preparing Raw Report
Report Format:
    - insights need to be in points 
    - Remember to focus mostly on recent quater and compare this with it's past 
    - And always give Numbers while comparing.
    - Each section should have atleast 5 insights points in it
    - Numbers should play dominant role in the Report    


Format Instructions:
{format_instructions}
Strictly follow this instruction

Example report:
{report}
This Example report is just for your reference, Don't use the "Example report" Anywhere while generating the report 

Raw Report:
{raw_report}

CSV Chart Data:
{chart_data}


Now begin to generate a quaterly report, with great insights.
"""
    prompt = PromptTemplate(
        template=template, input_variables=["report", "chart_data", "raw_report"],
        partial_variables={"format_instructions": format_instructions}
    )
    llm_chain = LLMChain(prompt=prompt, llm=gpt_4, verbose=True)
    responses = await llm_chain.apredict(report=Example_report, chart_data=Chart_data, raw_report=raw_report)
    try:
        responses = report_parser.parse(responses)
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        responses = await llm_chain.apredict(report=Example_report, chart_data=Chart_data, raw_report=raw_report)
        try:
            responses = report_parser.parse(responses)
        except Exception as e:
            print(f"An error occurred again during parsing: {e}")
            pass
        pass
    return responses


async def zero_shot_summary_generator(Chart_data):
    response_schemas = [
        ResponseSchema(name="Report", description="It Should Be a string."),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    # format_instructions = output_parser.get_format_instructions()
    format_instructions = report_parser.get_format_instructions()
    
    template = """
You are an AI Assistan helping to generate a Quarterly report with the given data 
The given data are,
    Textual Chart Data - it is a CSV like data which shows the quaterly performance of the different campaigns, Use those data to prepare a Quaterly report 
Report Format:
    - insights need to be in points with Rich detail.
    - Remember to focus mostly on recent quater and compare this with it's past 
    - Points need to be elaborate And always give Numbers while comparing or anywhere when you mention the data.
    - Each section should have atleast 5 insights points in it
    - Numbers should play dominant role in the Report    
Correlate the data in "Textual Chart Data" for better and deep insights from different Point of View.

Format Instructions:
{format_instructions}
Strictly follow this instruction

Textual Chart Data:
{chart_data}

Explain all the textual representation(Mostly CSV data) of chart data under "Textual Chart Data" and correlate those chart data in "Textual Chart Data" to get deep insights from all POV. 
your numbers should always match with actual number don't Hallucinate.

Now begin to generate a quaterly report with "Textual Chart Data", don't forget to correlate the chart data to get deep insights and don't miss any information.
"""
    prompt = PromptTemplate(
        template=template, input_variables=["chart_data"],
        partial_variables={"format_instructions": format_instructions}
    )
    llm_chain = LLMChain(prompt=prompt, llm=gpt_4, verbose=True)
    responses = await llm_chain.apredict(chart_data=Chart_data)
    try:
        responses = report_parser.parse(responses)
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        responses = await llm_chain.apredict(chart_data=Chart_data)
        try:
            responses = report_parser.parse(responses)
        except Exception as e:
            print(f"An error occurred again during parsing: {e}")
            pass
        pass
    return responses


async def latex_summary_merger(report, learnings, actions, chart, example_latex):

    template_single_shot = """ 
I'm Your boss and You are my Assistant(Professional Latex code developer), and your current project is to prepare a Report using the data given to you
The given data are,
    Raw Report - is the quaterly insights report
    Learnings - is the learing from the quaterly data
    Actions - is he action to take
    Chart Filename and Description - Under this you have Chart image Filename, and you have description about the chart image in Description and the data used to prepare chart is in "Data".
With these given data prepare a Latex code for complete report.
Format:
    - Use Wrap text format and Place the Chart images next(either leftg or right) to the Insights points from Raw Report for better understanding. 
    - Use all the images from "Chart Filename and Description" in the report so the report would have more insights.
    - Each section in each page. 
    - Learnings and Actions need to be in a table like format next to eachother and it should be in a new page at the end of the section.
    - Have '/Users/davidzoe/Downloads/pfizer.png' image from this path as footer on the left and page number to the right.
    - Report should be in Landscape style.


Single Shot Example:
    Raw Report:
        - Visits from the DAR video buy increased 44% QoQ from 67k visits in Q1 to 97k visits in Q2. This increase allowed for this channel to become a leading traffic driver for the quarter, although visits are not the primary KPI for this buy.
        - GRPs for this buy increased 33% QoQ, increasing the impressions for the buy, which could have attributed to the visit volume increase.
        - Overall, Working It creative drove stronger pages per visit on site and lower bounce rate, while the In This Together creative drove more visit volume to the site with a higher average visit duration.
    Learnings:
        - Digital Video: The introduction of the display digital video (non-DAR) placement in Q2 quickly drove more visits, making digital video a leading traffic driver. Tagging updates in Q3 allowed for WT to distinguish the difference between the DAR and non-DAR buys, and from that point until the end of 2021, WT saw 24% of total site visits were from display digital video. However, this share declined near the end of the year, possibly due to increased competition and possibly due to wear-out, with only one creative in market since May.
        - Additionally, 77% of visits from digital video were via desktop devices with this launch of the non-DAR buy. Visits from this buy were more engaged on the site. Desktop visitors overall were more likely to explore and engage with content on the site.
    Actions:
        - WT, Carat, and VMLY&R teams launched the "In This Together" spot alongside the currently live "Working It" spot as a test to understand how more than one creative performs in this new buy.
        - Current Status: Site side performance between the two creatives was similar overall. Team will continue to monitor as the buy shifts to include DEl-focused placements in Q3.

    Chart Filename and Description:
    Base directory - /Users/davidzoe/Downloads/Notebooks/Images
    
        Filename: Digital_Video_DAR_total.png
        Description: Table showing the metrices of visits, Page visits, Bounce Rate, Avg. Visit Duration and Download Rate of total.

        Filename: Digital_Video_DAR_working_it.png
        Description: Table showing the metrices of visits, Page visits, Bounce Rate, Avg. Visit Duration and Download Rate oo working it.

```latex
{example_latex}
```
Example is just for your reference don't use any data from example.


Raw Report:
{report}

Learnings:
{learnings}

Actions:
{actions}

Chart Filename and Description:
Base directory - /Users/davidzoe/Downloads/Notebooks/Images

{chart}

Your Latex code should be inside these
```latex
......
```

The report should be Simple and neat don't add any summary, conclusion anything like this. 

Important rule:
    - Don't leave any space for brevity or some other reason. complete everything by your own.
"""

    system_template = (
        """You are an AI Assistant(Professional Latex code developer), and your current task is to prepare a Report using the data given to you
The given data are,
    Raw Report - is the quaterly insights report
    Learnings - is the learing from the quaterly data
    Actions - is he action to take
    Chart Filename and Description - Under this you have Chart image Filename, and you have description about the chart image in Description and the data used to prepare chart is in "Data".
With these given data prepare a Latex code for complete report.
Format:
    - Insight need to be in bullet points.
    - Use Wrap text format and Place the Chart images next(either leftg or right) to the Insights points from Raw Report for better understanding. 
    - Each Section should be in a new page and have a heading for that. 
    - Learnings and Actions need to be in a table like format next to eachother and it should be in a new page at the end of the section.
    - Have '/Users/davidzoe/Downloads/pfizer.png' image from this path as footer on the left and page number to the right.
    - Report should be in Landscape style.
    - Place all the image in the appropriate places.

The report should be Simple and neat don't add any summary, conclusion anything like this. And it should have all the things mentioned above

Important rule:
    - Don't leave any space for brevity or some other reason. complete everything by your own.

"""
    )
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_template = """Raw Report:
{report}

Learnings:
{learnings}

Actions:
{actions}

Chart Filename and Description:
Base directory - /Users/davidzoe/Downloads/Notebooks/Images

{chart}

Your Latex code should be inside these
```latex
......
```

"""
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )
    
    template = """
You are an AI Assistant(Professional Latex code developer), and your current task is to prepare a Report using the data given to you
The given data are,
    Raw Report - is the quaterly insights report
    Learnings - is the learing from the quaterly data
    Actions - is he action to take
    Chart Filename and Description - Under this you have Chart image Filename, and you have description about the chart image in Description and the data used to prepare chart is in "Data".
With these given data prepare a Latex code for complete report.
Format:
    - Insight need to be in bullet points.
    - Use Wrap text format and Place the Chart images next(either leftg or right) to the Insights points from Raw Report for better understanding. 
    - Each Section should be in a new page. 
    - Learnings and Actions need to be in a table like format next to eachother and it should be in a new page at the end of the section.
    - Have '/Users/davidzoe/Downloads/pfizer.png' image from this path as footer on the left and page number to the right.
    - Report should be in Landscape style.

Raw Report:
{report}

Learnings:
{learnings}

Actions:
{actions}

Chart Filename and Description:
Base directory - /Users/davidzoe/Downloads/Notebooks/Images

{chart}

Your Latex code should be inside these
```latex
......
```

The report should be Simple and neat don't add any summary, conclusion anything like this. And it should have all the things mentioned above

Important rule:
    - Don't leave any space for brevity or some other reason. complete everything by your own.

"""
    prompt = PromptTemplate(
        template=template, input_variables=["report", "learnings", "actions", "chart", "example_latex"],
        # partial_variables={"format_instructions": format_instructions}
    )
    llm_chain = LLMChain(prompt=chat_prompt, llm=gpt_4, verbose=True)
    # responses = await llm_chain.apredict(report=report, learnings=learnings, actions=actions, chart=chart, example_latex=example_latex)
    responses = await llm_chain.arun({"report":report, "learnings":learnings, "actions":actions, "chart":chart, "example_latex":example_latex})

    print(responses)
    pattern = re.compile(r'```latex(.*?)```', re.DOTALL)
    matches = pattern.findall(responses)
    
    # Writing extracted strings to a .tex file
    with open('output.tex', 'w') as tex_file:
        for match in matches:
            tex_file.write(match.strip() + '\n')
    return matches[0]




In [52]:
example_latex = r"""    
\documentclass[landscape]{article}
\usepackage{graphicx}
\usepackage{wrapfig}
\usepackage{lscape}
\usepackage{fancyhdr}
\usepackage{geometry}
\usepackage[table]{xcolor}
\usepackage{longtable}

% Set page margins
\geometry{left=1in,right=1in,top=1in,bottom=1in}

% Header and footer settings
\pagestyle{fancy}
\fancyhf{}
\lfoot{\includegraphics[height=1cm]{/Users/davidzoe/Downloads/pfizer.png}}
\rfoot{\thepage}
\renewcommand{\headrulewidth}{0pt}
\renewcommand{\footrulewidth}{0pt}

% Define colors for table rows
\definecolor{gray1}{gray}{0.9}
\definecolor{gray2}{gray}{0.8}

\begin{document}

% Quarterly Performance Insights Section
\section*{Quarterly Performance Insights:}

% Paid Social Digital Video DAR - Total
\subsection*{1. Paid Social Digital Video DAR - Total}
\begin{wrapfigure}{r}{0.5\textwidth}
  \includegraphics[width=0.48\textwidth]{/Users/davidzoe/Downloads/Notebooks/Images/Digital_Video_DAR_total.png}
\end{wrapfigure}
    - Visits from the DAR video buy increased 44Percentage QoQ from 67k visits in Q1 to 97k visits in Q2. This increase allowed for this channel to become a leading traffic driver for the quarter, although visits are not the primary KPI for this buy.
    - GRPs for this buy increased 33Percentage QoQ, increasing the impressions for the buy, which could have attributed to the visit volume increase.

\clearpage

% Paid Social Digital Video DAR - Working it
\subsection*{2. Paid Social Digital Video DAR - Working it}
\begin{wrapfigure}{l}{0.5\textwidth}
  \includegraphics[width=0.48\textwidth]{/Users/davidzoe/Downloads/Notebooks/Images/Digital_Video_DAR_working_it.png}
\end{wrapfigure}
    - Overall, Working It creative drove stronger pages per visit on site and lower bounce rate, while the In This Together creative drove more visit volume to the site with a higher average visit duration.

\clearpage

% Paid Social Digital Video DAR - Total
\subsection*{1. Paid Social Digital Video DAR - Total}
\begin{wrapfigure}{r}{0.5\textwidth}
  \includegraphics[width=0.48\textwidth]{/Users/davidzoe/Downloads/Notebooks/Images/Digital_Video_DAR_total.png}
\end{wrapfigure}
    - Visits from the DAR video buy increased 44Percentage QoQ from 67k visits in Q1 to 97k visits in Q2. This increase allowed for this channel to become a leading traffic driver for the quarter, although visits are not the primary KPI for this buy.
    - GRPs for this buy increased 33Percentage QoQ, increasing the impressions for the buy, which could have attributed to the visit volume increase.

\clearpage

% Paid Social Digital Video DAR - Working it
\subsection*{2. Paid Social Digital Video DAR - Working it}
\begin{wrapfigure}{l}{0.5\textwidth}
  \includegraphics[width=0.48\textwidth]{/Users/davidzoe/Downloads/Notebooks/Images/Digital_Video_DAR_working_it.png}
\end{wrapfigure}
    - Overall, Working It creative drove stronger pages per visit on site and lower bounce rate, while the In This Together creative drove more visit volume to the site with a higher average visit duration.

\clearpage

% Paid Social Digital Video DAR - Total
\subsection*{1. Paid Social Digital Video DAR - Total}
\begin{wrapfigure}{r}{0.5\textwidth}
  \includegraphics[width=0.48\textwidth]{/Users/davidzoe/Downloads/Notebooks/Images/Digital_Video_DAR_total.png}
\end{wrapfigure}
    - Visits from the DAR video buy increased 44Percentage QoQ from 67k visits in Q1 to 97k visits in Q2. This increase allowed for this channel to become a leading traffic driver for the quarter, although visits are not the primary KPI for this buy.
    - GRPs for this buy increased 33Percentage QoQ, increasing the impressions for the buy, which could have attributed to the visit volume increase.

\clearpage

% Paid Social Digital Video DAR - Working it
\subsection*{2. Paid Social Digital Video DAR - Working it}
\begin{wrapfigure}{l}{0.5\textwidth}
  \includegraphics[width=0.48\textwidth]{/Users/davidzoe/Downloads/Notebooks/Images/Digital_Video_DAR_working_it.png}
\end{wrapfigure}
    - Overall, Working It creative drove stronger pages per visit on site and lower bounce rate, while the In This Together creative drove more visit volume to the site with a higher average visit duration.

\clearpage


% Table for Learnings and Actions
\begin{longtable}{|L|A|}
\hline
\rowcolor{gray!25}
\textbf{Learnings} & \textbf{Actions} \\
\hline
\endfirsthead
\hline
\rowcolor{gray!25}
\textbf{Learnings} & \textbf{Actions} \\
\hline
\endhead
\hline
\endfoot
\hline
\endlastfoot

Digital Video: The introduction of the display digital video (non-DAR) placement in Q2 quickly drove more visits, making digital video a leading traffic driver. Tagging updates in Q3 allowed for WT to distinguish the difference between the DAR and non-DAR buys, and from that point until the end of 2021, WT saw 24percentage of total site visits were from display digital video. However, this share declined near the end of the year, possibly due to increased competition and possibly due to wear-out, with only one creative in market since May. & WT, Carat, and VMLY&R teams launched the "In This Together" spot alongside the currently live "Working It" spot as a test to understand how more than one creative performs in this new buy. \\
\hline
Additionally, 77percentage of visits from digital video were via desktop devices with this launch of the non-DAR buy. Visits from this buy were more engaged on the site. Desktop visitors overall were more likely to explore and engage with content on the site. & Current Status: Site side performance between the two creatives was similar overall. Team will continue to monitor as the buy shifts to include DEl-focused placements in Q3. \\
\end{longtable}

\end{document}
"""


In [10]:
zero_shot_report = asyncio.run(zero_shot_summary_generator(Chart_data=Paid_Social_Current))




> Entering new LLMChain chain...
Prompt after formatting:

You are an AI Assistan helping to generate a Quarterly report with the given data 
The given data are,
    Textual Chart Data - it is a CSV like data which shows the quaterly performance of the different campaigns, Use those data to prepare a Quaterly report 
Report Format:
    - insights need to be in points with Rich detail.
    - Remember to focus mostly on recent quater and compare this with it's past 
    - Points need to be elaborate And always give Numbers while comparing or anywhere when you mention the data.
    - Each section should have atleast 5 insights points in it
    - Numbers should play dominant role in the Report    
Correlate the data in "Textual Chart Data" for better and deep insights from different Point of View.

Format Instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "descripti

In [11]:
few_shot_report = asyncio.run(few_shot_summary_generator(Example_report=actual_dict[2]["Report"], 
                                                  Chart_data=Paid_Social_Current, 
                                                  raw_report=zero_shot_report.Report))




> Entering new LLMChain chain...
Prompt after formatting:

You are an AI Assistant helping user to generate a Quarterly report with the given data.
The given data are,
    Example report - it is a part of a Quaterly report, just given to you for your reference and understanding 
    Raw Report - this has mostly all the detail you need to write a quaterly report
    CSV Chart Data - is where you can find all the raw data(CSV formatted), this data is used while preparing Raw Report
Report Format:
    - insights need to be in points 
    - Remember to focus mostly on recent quater and compare this with it's past 
    - And always give Numbers while comparing.
    - Each section should have atleast 5 insights points in it
    - Numbers should play dominant role in the Report    


Format Instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of st

In [24]:
report_validation = asyncio.run(report_matcher(actual_report=Paid_Social_Current, generated_report=few_shot_report.Report))



> Entering new LLMChain chain...
Prompt after formatting:
Your task is to asses the generated report with actual report
our goal is to have generate similar report like actual report so give valuable feedback
You will be given both generated and actual report with that give your response

Format Instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Match": {"title": "Match", "description": "It Should Be 'True' if it is a good match else it need to be 'False'.", "type": "boolean"}, "Similarities": {"title": "Similarities", "description": "It Should Be similarity be

In [25]:
print(report_validation.Differences)

The actual report provides raw data in a tabular format, while the generated report provides a narrative analysis of the data. The actual report includes a 'DCM_FLOODLIGHTS_UNIQUE_VISITS' column which is not mentioned in the generated report. The generated report includes interpretations and suggestions for strategy adjustments, which are not present in the actual report. The generated report also ranks campaigns by performance, which is not explicitly done in the actual report. The actual report includes precise figures, whereas the generated report rounds off the numbers and converts them into a more readable format (e.g., $12,700.42 instead of 12700.4159). The generated report omits the 'I am Still Me Driving' campaign and includes a 'Quarterly Performance Report Insights' title, which is not present in the actual report.


In [26]:
learning_actions = asyncio.run(learning_and_action_gen(key_takeaways=few_shot_report.Report))




> Entering new LLMChain chain...
Prompt after formatting:
Your task is use the key takeaway and with that generate Learnings and Actions to the business
You will be given example Learnigs, example Actions and example key takeaways use that as a reference
Number points of learnings should always match number points of actions.

Format Instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Learnings": {"title": "Learnings", "description": "It Should Be a Learning of from key Takeaways.", "type": "array", "items": {}}, "Actions": {"title": "Actions", "description": "I

In [28]:
import json

# Specify the path to your JSON file
json_file_path = "/Users/davidzoe/Downloads/Notebooks/Images/charts_data.json"

# Open and read the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)


In [54]:
# chart_location = [{"Filename": d["Filename"], "Description": d["Description"], "Data": d["Data"]} for d in data]

# chart_location_formated = ""

# for index, d in enumerate(chart_location):
#     chart_location_formated += f"  Filename: {d['Filename']}\n"
#     chart_location_formated += f"  Description: {d['Description']}\n"
#     chart_location_formated += f"  Data: {d['Data']}\n\n"


chart_location_formatted = ""
for i in range(len(data["Filename"])):
    chart_location_formatted += f"Filename: {data['Filename'][i]}\n"
    chart_location_formatted += f"Description: {data['Description'][i]}\n"
    for key, values in data['Data'][i].items():
        chart_location_formatted += f"{key}:\n"
        for sub_key, value in values.items():
            chart_location_formatted += f"  {sub_key}: {value}\n"
    chart_location_formatted += "\n"







In [56]:
summary = asyncio.run(latex_summary_merger(report=few_shot_report.Report, learnings=learning_actions.Learnings, actions=learning_actions.Learnings, chart=chart_location_formatted, example_latex=example_latex))




> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI Assistant(Professional Latex code developer), and your current task is to prepare a Report using the data given to you
The given data are,
    Raw Report - is the quaterly insights report
    Learnings - is the learing from the quaterly data
    Actions - is he action to take
    Chart Filename and Description - Under this you have Chart image Filename, and you have description about the chart image in Description and the data used to prepare chart is in "Data".
With these given data prepare a Latex code for complete report.
Format:
    - Insight need to be in bullet points.
    - Use Wrap text format and Place the Chart images next(either leftg or right) to the Insights points from Raw Report for better understanding. 
    - Each Section should be in a new page and have a heading for that. 
    - Learnings and Actions need to be in a table like format next to eachother and it should be in a new page at t

### Generated Report Matcher

In [13]:
# Define your desired data structure.
class Report_Matcher(BaseModel):
    Match: bool = Field(description="It Should Be 'True' if it is a good match else it need to be 'False'.")
    Similarities: str = Field(description="It Should Be similarity between Actual report and Generated report.")
    Differences: str = Field(description="It Should Be a differences between Actual report and Generated report.")
    
report_matcher_parser = PydanticOutputParser(pydantic_object=Report_Matcher)


In [14]:
async def report_matcher(actual_report, generated_report):
    response_schemas = [
        ResponseSchema(name="Match", description="It Should Be 'Yes' if it is a good match else it need to be 'No'.", type="str"),
        ResponseSchema(name="Similarities", description="It Should Be similarity between Actual report and Generated report.", type="str"),
        ResponseSchema(name="Differences ", description="It Should Be a differences between Actual report and Generated report.", type="str"),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    
    # format_instructions = output_parser.get_format_instructions()
    format_instructions = report_matcher_parser.get_format_instructions()
    
    template = """Your task is to asses the generated report with actual report
our goal is to have generate similar report like actual report so give valuable feedback
You will be given both generated and actual report with that give your response

Format Instructions:
{format_instructions}

Actual report:
{actual_report}

Generated report:
{generated_report}
"""
    prompt = PromptTemplate(
        template=template, input_variables=["actual_report", "generated_report"],
        partial_variables={"format_instructions": format_instructions}
    )
    llm_chain = LLMChain(prompt=prompt, llm=gpt_4, verbose=True)
    responses = await llm_chain.apredict(actual_report=actual_report, generated_report=generated_report)
    try:
        responses = report_matcher_parser.parse(responses)
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        responses = await llm_chain.apredict(actual_report=actual_report, generated_report=generated_report)
        try:
            responses = report_matcher_parser.parse(responses)
        except Exception as e:
            print(f"An error occurred again during parsing: {e}")
            pass
        pass

    return responses

### Learnings And Actions

In [15]:
# Define your desired data structure.
class Learnings_and_actions(BaseModel):
    Learnings: List = Field(description="It Should Be a Learning of from key Takeaways.")
    Actions: List = Field(description="It Should Be Actions to take to improve the situation.")

learning_and_action_parser = PydanticOutputParser(pydantic_object=Learnings_and_actions)


In [16]:
async def learning_and_action_gen(key_takeaways):
    response_schemas = [
        ResponseSchema(name="Learnings", description="It Should Be a Learning of from key Takeaways.", type="list"),
        ResponseSchema(name="Actions", description="It Should Be Actions to take to improve the situation.", type="list"),
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    
    # format_instructions = output_parser.get_format_instructions()
    format_instructions = learning_and_action_parser.get_format_instructions()
    
    template = """Your task is use the key takeaway and with that generate Learnings and Actions to the business
You will be given example Learnigs, example Actions and example key takeaways use that as a reference
Number points of learnings should always match number points of actions.

Format Instructions:
{format_instructions}

Example Key Takeaways:

- Visits from the DAR video buy increased 44% QoQ from 67k visits in Q1 to 97k visits in Q2. This increase allowed for this channel to become a leading traffic driver for the quarter, although visits are not the primary KPI for this buy.
- GRPs for this buy increased 33% QoQ, increasing the impressions for the buy, which could have attributed to the visit volume increase.
- Overall, Working It creative drove stronger pages per visit on site and lower bounce rate, while the In This Together creative drove more visit volume to the site with a higher average visit duration.

Example Learnigs:

Digital Video Current and Past Learnings:
- Digital Video: The introduction of the display digital video (non-DAR) placement in Q2 quickly drove more visits, making digital video a leading traffic driver. Tagging updates in Q3 allowed for WT to distinguish the difference between the DAR and non-DAR buys, and from that point until the end of 2021, WT saw 24% of total site visits were from display digital video. However, this share declined near the end of the year, possibly due to increased competition and possibly due to wear-out, with only one creative in market since May.
- Additionally, 77% of visits from digital video were via desktop devices with this launch of the non-DAR buy. Visits from this buy were more engaged on the site. Desktop visitors overall were more likely to explore and engage with content on the site.

Example Actions:

Digital Actions and Looking Ahead:
- • WT, Carat, and VMLY&R teams launched the "In This Together" spot alongside the currently live Working It" spot as a test to understand how more than one creative performs in this new buy.
- Current Status: Site side performance between the two creatives was similar overall. Team will continue to monitor as the buy shifts to include DEl-focused placements in Q3.


Key Takeaways:
{key_takeaways}
"""
    prompt = PromptTemplate(
        template=template, input_variables=["key_takeaways"],
        partial_variables={"format_instructions": format_instructions}
    )
    llm_chain = LLMChain(prompt=prompt, llm=gpt_4, verbose=True)
    responses = await llm_chain.apredict(key_takeaways=key_takeaways)
    try:
        responses = learning_and_action_parser.parse(responses)
    except Exception as e:
        print(f"An error occurred during parsing: {e}")
        responses = await llm_chain.apredict(key_takeaways=key_takeaways)
        try:
            responses = learning_and_action_parser.parse(responses)
        except Exception as e:
            print(f"An error occurred again during parsing: {e}")
            pass
        pass

    return responses

### Full Generation

In [559]:
async def report_gen_and_check(data):
    global actual_dict
    Report_data = copy.deepcopy(data)

    zero_shot_tasks = [zero_shot_summary_generator(Chart_data=data["Chart"]) for data in Report_data] 

    chart_tasks = [chart_generator(chart_data=data["Chart"]) for data in Report_data]

    zero_shot_reports, charts = await asyncio.gather(
        asyncio.gather(*zero_shot_tasks),
        asyncio.gather(*chart_tasks)
    )
    print(f"Type{type(charts)}\n{charts}")
    for data, zero_shot_report in zip(Report_data, zero_shot_reports):
        data["Zero_shot_report"] = zero_shot_report.Report
        
    for data, chart in zip(Report_data, charts):
        data["Chart_gen"] = chart
        exec(chart)

    few_shot_tasks = [few_shot_summary_generator(Example_report=actual_dict[2]["Report"],
                                          Chart_data=data["Chart"], 
                                          raw_report=data["Zero_shot_report"])for data in Report_data]
    few_shot_reports  = await asyncio.gather(*few_shot_tasks)
    
    for data, few_shot_report in zip(Report_data, few_shot_reports):
        data["Gen_Report"] = few_shot_report.Report         
            
    # Run report_matcher tasks concurrently
    report_check_tasks = [report_matcher(actual_report=data["Report"], generated_report=data["Gen_Report"]) for data in Report_data]
    # report_check = await asyncio.gather(*report_check_tasks)

    # Run report_matcher tasks concurrently
    learning_action_tasks = [learning_and_action_gen(key_takeaways=data["Gen_Report"]) for data in Report_data]
    # learnings_actions = await asyncio.gather(*learning_action_tasks)

    report_check, learnings_actions = await asyncio.gather(
        asyncio.gather(*report_check_tasks),
        asyncio.gather(*learning_action_tasks)
    )
    # Update each dictionary in Report_data with Check
    for data, check_result in zip(Report_data, report_check):
        data["Check"] = {"Match": check_result.Match, "Similarities": check_result.Similarities,"Differences": check_result.Differences}

    # Update each dictionary in Report_data with Check
    for data, learnings_action in zip(Report_data, learnings_actions):
        data["L_A"] = {"Learnings": learnings_action.Learnings, "Actions": learnings_action.Actions}

    # Specify the path to your JSON file
    json_file_path = "/Users/davidzoe/Downloads/Notebooks/Images/charts_data.json"
    
    # Open and read the JSON file
    with open(json_file_path, 'r') as file:
        data = json.load(file)
        
    # Run latex_summary_merger tasks concurrently
    latex_code_tasks = [latex_summary_merger(report=data["Gen_Report"], actions=data["L_A"]["Actions"], learnings=data["L_A"]["Learnings"], chart=data) for data in Report_data]
    latex_codes = await asyncio.gather(*latex_code_tasks)

    # Update each dictionary in Report_data with Check
    for data, latex_code in zip(Report_data, latex_codes):
        data["Latex_code"] = latex_code
        
    return Report_data




In [560]:
updated_data = asyncio.run(report_gen_and_check(individual_dict))



> Entering new LLMChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an AI Assistan helping to generate a Quarterly report with the given data 
The given data are,
    Textual Chart Data - it is a CSV like data which shows the quaterly performance of the different campaigns, Use those data to prepare a Quaterly report 
Report Format:
    - insights need to be in points with Rich detail.
    - Remember to focus mostly on recent quater and compare this with it's past 
    - Points need to be elaborate And always give Numbers while comparing or anywhere when you mention the data.
    - Each section should have atleast 5 insights points in it
    - Numbers should play dominant role in the Report    
Correlate the data in "Textual Chart Data" for better and deep insights from different Point of View.

Format Instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": 

In [561]:
print(updated_data[0])

{'Chart': '\n1. Paid Social Facebook&Instagram - Reach Optimized\n\n\nKPI: Monthly Reached\nPlanned: 30%\nActualized: 36%\nApril: 37%, May: 35%, June: - \n\nGlobal:\n\nQuaterly Impressions:\nWeek of Actualdate [2022], Doctor Discussion Guide, First Of Its Kind - 2021, Live In Your Moment-2021, MP New 125K Claim, Newly Diagnosed-2021\nMar 27, 400, 100, 20, 180, 10\nApr 3, 380, 120, 35, 100, 10\t\nApr 10, 390, 115, 65, 55, 10\t\nApr 17, 420, 110, 35, 70, 10\t\t\t\nApr 24, 410, 180, 20, 85, 10\t\t\nMay 1, 490,  100, 15, 105, 10\t\nMay 8, 480,\t120, 15, 120, 10\nMay 15, 370, 90, 15, 90, 10\n\nQ2 Creative Performance:\nAd Description, Impressions, % of IMP\nDoctor Discussion Guide, 58391183, 56%\nFirst of its Kind - 2021, 18520560, 18%\nMP New 125K Claim, 15768707, 15%\nLive In Your Moment - 2021, 7109039, 7%\nNewly Diagnosed - 2021, 4055879, 4%\n\nMixed Placements Creative Comparison - Daily Visits:\nDate, Doctor Discussion Guide, First of Its Kind, Living in Your Moment, Newly Diagnosed, 

### Chart Creation

In [17]:
async def chart_generator(chart_data):

    template = """You are an Professional Python Developer, who mastered chreating charts using Plotly
Your current project is to produce Python script using Plotly to prepare chart for the give textualized data(Mostly CSV)
The given data are,
    Textualized CSV Data - is where you can find multiple data(Mostly Quaterly based) which need to be converted into individual chart.

Things need to remember while writing the plotly python script for chart:
    - You can use these charts Tables(If the data points represent different type like numeric in one row percentage in other row), line charts, multiple line charts, pie charts, area charts, and stacked area charts, bar diagrams, stacked bar diagrams. Choose the chart wisely based on the data.
    - And you need to save those chart as a '.png' file in ths directory "/Users/davidzoe/Downloads/Notebooks/Images"
    - Filename should be heading of the chart with title and campaign, so it can be identified easily.
    - No need for creating any function, just save those data in different variables in required type and prepare a chart and save the chart as .png.
    - Write complete errorless python script don't leave space for me to complete cause i don't know anything about it.

Json File:
    - Save a json file which should have three key "Filename", "Description" and "Data" for each image.
    - "Filename" should have all the saved image(Chart) file name.
    - "Description" should be the description about the saved image, mentioning its campigns and more.
    - "Data" should have the data used to prepare chart.



Your Python code need to be inside ths for sure
```python
......
```

Textualized CSV Data:
{chart_data}


First save all the CSV like data in a variable as a dataframe or list or dict like object for easy use, don't omit any data for brevity or some other reason, then start to prepare individual chart. Then save those chart and prepare Json file.


Important rule:
    - Don't ever stop in the middle or leave space for me to complete. You are supposed to give complete code.
"""

    prompt = PromptTemplate(
        template=template, input_variables=["chart_data"],
    )
    llm_chain = LLMChain(prompt=prompt, llm=gpt_4, verbose=True)

    responses = await llm_chain.apredict(chart_data=chart_data)
    print(responses)
    pattern = re.compile(r'```python(.*?)```', re.DOTALL)
    responses = pattern.findall(responses)
    print(responses[0])


    return responses[0]
    

In [21]:
async def chart_bulk_curation(Report_data):
    # Run report_matcher tasks concurrently
    # chart_tasks = [chart_generator(chart_data=data["Chart"]) for data in Report_data]
    chart_tasks = [chart_generator(chart_data=Report_data)]

    charts = await asyncio.gather(*chart_tasks)

    # Update each dictionary in Report_data with Check
    # for data, check_result in zip(Report_data[start:end], report_check):
    for data, chart in zip(Report_data, charts):   
        try:
            data["Chart_gen"] = chart["Code"]

        except Exception as e:
            print("An error occured While Parsing so it's kept as a string.")
            data["Chart_gen"] = chart

    return Report_data


In [22]:
updated_data = asyncio.run(chart_bulk_curation(Paid_Social_Current))



> Entering new LLMChain chain...
Prompt after formatting:
You are an Professional Python Developer, who mastered chreating charts using Plotly
Your current project is to produce Python script using Plotly to prepare chart for the give textualized data(Mostly CSV)
The given data are,
    Textualized CSV Data - is where you can find multiple data(Mostly Quaterly based) which need to be converted into individual chart.

Things need to remember while writing the plotly python script for chart:
    - You can use these charts Tables(If the data points represent different type like numeric in one row percentage in other row), line charts, multiple line charts, pie charts, area charts, and stacked area charts, bar diagrams, stacked bar diagrams. Choose the chart wisely based on the data.
    - And you need to save those chart as a '.png' file in ths directory "/Users/davidzoe/Downloads/Notebooks/Images"
    - Filename should be heading of the chart with title and campaign, so it can be ident

TypeError: 'str' object does not support item assignment

In [ ]:
print(updated_data[0]["Chart_gen"])

In [ ]:
exec(updated_data[0]["Chart_gen"])

In [23]:
import pandas as pd
import plotly.express as px
import json
import os

# Create a DataFrame from the provided CSV data
data = {
    "CREATIVE_DESCRIPTION": [
        "BCG Traffic", "Class Leadership 2023", "Doctor Discussion Guide",
        "Financial Support 2022", "I am Still Me Driving", "I am Still Me Gaming",
        "Newly Diagnosed", "Not Traditional Chemo", "Real World Evidence",
        "Ways to Connect", "Webinar Driver"
    ],
    "CLICKS": [12.0, 1325.0, 12322.0, 3333.0, 38572.0, 50400.0, 367.0, 673.0, 14616.0, 2.0, 31870.0],
    "IMPRESSIONS": [2.0, 151677.0, 1119076.0, 235334.0, 4053184.0, 7138341.0, 52055.0, 67812.0, 1478048.0, 0.0, 3218559.0],
    "MEDIA_COST": [0.0000, 1305.8217, 12700.4159, 3402.1164, 38555.6432, 52971.4538, 370.9775, 607.0957, 14793.5061, 0.0000, 31554.6514],
    "DCM_FLOODLIGHTS_UNIQUE_VISITS": [0.0, 4.0, 12.0, 3.0, 33.0, 36.0, 3.0, 4.0, 26.0, 0.0, 66.0],
    "VIDEO_VIEWS": [0.0, 14399.0, 202881.0, 49794.0, 670102.0, 1044314.0, 5640.0, 9344.0, 0.0, 0.0, 0.0]
}
df = pd.DataFrame(data)

# Directory to save images
image_dir = "/Users/davidzoe/Downloads/Notebooks/Images"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# JSON data
json_data = {
    "Filename": [],
    "Description": [],
    "Data": []
}

# Function to save chart as PNG and update JSON data
def save_chart(fig, filename, description, data):
    image_path = os.path.join(image_dir, filename + '.png')
    fig.write_image(image_path)
    json_data["Filename"].append(filename + '.png')
    json_data["Description"].append(description)
    json_data["Data"].append(data)

# Create and save charts
for column in df.columns[1:]:
    fig = px.bar(df, x='CREATIVE_DESCRIPTION', y=column, title=f"{column} by Campaign")
    filename = f"{column}_by_Campaign"
    description = f"Bar chart showing {column} for each campaign."
    save_chart(fig, filename, description, df[['CREATIVE_DESCRIPTION', column]].to_dict())

# Save JSON data to a file
json_path = os.path.join(image_dir, 'charts_data.json')
with open(json_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print("Charts and JSON data have been saved successfully.")


Charts and JSON data have been saved successfully.
